# Notebook 3 - Feature Engineering

In this notebook we consider various feature engineering techniques. Some of these were suggested in the forums for TPS 12 as well as in the original forest cover type competition.

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 12

In [2]:
import numpy as np
import pandas as pd
import math
import scipy
import time
import pyarrow
import gc

# Model evaluation
from functools import partial
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier

# Plotting
import matplotlib
import seaborn as sns
from matplotlib import pyplot as plt

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Prepare Data

In [3]:
%%time

# Load original data
original = pd.read_feather('../data/original.feather')

# Label Encode
old_encoder = LabelEncoder()
original["Cover_Type"] = old_encoder.fit_transform(original["Cover_Type"])
y_train = original['Cover_Type'].iloc[:15119]
y_test = original['Cover_Type'].iloc[15119:]

# Get feature columns
features = [x for x in original.columns if x not in ['Id','Cover_Type'] and not x.startswith('Soil')]

# Data structures for summary scores
bagging_scores = list()
extratrees_scores = list()
adaboost_scores = list()
random_scores = list()

Wall time: 51.7 ms


# Scoring Function

In [4]:
def train_original(sklearn_model, processing = None, verbose = False):
    
    og_features = features + ['Soil_Type']
    
    # Feature Engineering
    if processing:
        X = processing(original)
    
    # Original Training/Test Split
    X_temp = X.iloc[:15119]
    X_test = X.iloc[15119:]
    y_temp = X['Cover_Type'].iloc[:15119]
    y_test = X['Cover_Type'].iloc[15119:]
    
    new_features = [x for x in X_temp.columns if x not in og_features and x not in ['Id','Cover_Type']]
    
    results = list()
    
    # Iterate over columns
    for new_feature in [None, *new_features]:
        
        # Store the out-of-fold predictions
        test_preds = np.zeros((X_test.shape[0],7))
        oof_preds = np.zeros((X_temp.shape[0],))
        scores, times = np.zeros(NUM_FOLDS), np.zeros(NUM_FOLDS)
        
        if new_feature:
            cols = og_features + [new_feature]
        else:
            cols = og_features

        # Stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
        for fold, (train_idx, valid_idx) in enumerate(skf.split(X_temp,y_temp)):

            # Training and Validation Sets
            X_train, X_valid = X_temp[cols].iloc[train_idx], X_temp[cols].iloc[valid_idx]
            y_train, y_valid = y_temp.iloc[train_idx], y_temp.iloc[valid_idx]

            # Create model
            start = time.time()
            model = clone(sklearn_model)
            model.fit(X_train, y_train)

            # validation and test predictions
            valid_preds = np.ravel(model.predict(X_valid))
            oof_preds[valid_idx] = valid_preds
            test_preds += model.predict_proba(X_test[cols])

            # Save scores and times
            scores[fold] = accuracy_score(y_valid, valid_preds)
            end = time.time()
            times[fold] = end-start
            time.sleep(0.5)

        test_preds = np.argmax(test_preds, axis = 1)
        test_score = accuracy_score(y_test, test_preds)
        oof_f1 = f1_score(y_temp, oof_preds, average = 'macro')
        if verbose:
            print('\n'+model.__class__.__name__)
            if new_feature: 
                print('Added Feature:', new_feature)
            else:
                print('Baseline')
            print(f'Training Time: {round(times.sum(), 2)}s')
            print("OoF F1 Score:", oof_f1)
            print("Train Accuracy:", round(scores.mean(), 5))
            print('Test Accuracy:', round(test_score, 5))
        
        
        if new_feature:
            results.append((
                new_feature, scores.mean(), test_score, oof_f1, times.sum()
            ))
        else:
            results.append((
                'Baseline', scores.mean(), test_score, oof_f1, times.sum()
            ))
    
    return results

# Model

In [5]:
# ExtraTrees Classifier
extratrees = ExtraTreesClassifier(
    n_jobs = -1,
    random_state = RANDOM_SEED,
    max_features = None,
)

# Feature Engineering

In [6]:
def feature_engineering(data):
    df = data.copy()
    
    # Get columns
    shade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
    soil_features = [f'Soil_Type{i}' for i in range(1,41)]
    
    
    # Use float64 for calculations
    for col, dtype in df.dtypes.iteritems():
        if dtype.name.startswith('float'):
            df[col] = df[col].astype('float64')
    
    # Replace soil type columns with categoricals
    df['Soil_Type'] = 0
    for i in range(1,41):
        df['Soil_Type'] += i*df[f'Soil_Type{i}']
        
    # Soil Interactions
    df['Soil_12_32'] = df['Soil_Type32'] + df['Soil_Type12']
    df['Soil_Type23_22_32_33'] = df['Soil_Type23'] + df['Soil_Type22'] + df['Soil_Type32'] + df['Soil_Type33']
    
    # Aspect Features
    df['Aspect_360'] = df['Aspect'] % 360
    df['Aspect_Sine'] = (df['Aspect']* np.pi / 180).apply(np.sin)
    df['Aspect_Alt'] = (df['Aspect']-180).where(df['Aspect']+180 > 360, df['Aspect'] + 180)
    
    # Log features
    df['Horizontal_Distance_To_Roadways_Log'] = [math.log(v+1) for v in df['Horizontal_Distance_To_Roadways']]
    df['Horizontal_Distance_To_Hydrology_Log'] = [math.log(v+1) for v in df['Horizontal_Distance_To_Hydrology']]
        
    # Binned Elevation
    df['Elev_Binned'] = [math.floor(v/50.0) for v in df['Elevation']]
    
    # Water Distance
    df["Hydro_Taxicab"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
    df["Hydro_Euclid"] = (df["Horizontal_Distance_To_Hydrology"]**2 + np.abs(df["Vertical_Distance_To_Hydrology"])**2)**0.5
    df['Water_Direction'] = df['Vertical_Distance_To_Hydrology'].apply(np.sign)
    df['Water Elevation'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    
    
    # Shade features
    df["Hillshade_Avg"] = df[shade_features].mean(axis=1)
    df['Hillshade_Range'] = df[shade_features].max(axis=1) - df[shade_features].min(axis=1)
    df['Hillshade'] = df[shade_features].sum(axis=1)
    
    # Distance Interactions
    df['Hydro_Fire_1'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    
    # Elevation Interactions
    df['EHiElv'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    df['EViElv'] = df['Vertical_Distance_To_Hydrology'] * df['Elevation']
    df['EVDtH'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    df['EHDtH'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    df['Elev_3Horiz'] = df['Elevation'] + df['Horizontal_Distance_To_Roadways']  + df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Hydrology']
    df['Elev_Road_1'] = df['Elevation'] + df['Horizontal_Distance_To_Roadways']
    df['Elev_Road_2'] = df['Elevation'] - df['Horizontal_Distance_To_Roadways']
    df['Elev_Fire_1'] = df['Elevation'] + df['Horizontal_Distance_To_Fire_Points']
    df['Elev_Fire_2'] = df['Elevation'] - df['Horizontal_Distance_To_Fire_Points']
    df['Elev_Hillshade_1'] = df['Elevation'] - df['Hillshade']
    df['Elev_Hillshade_2'] = df['Elevation'] + df['Hillshade']
    
    # ELU soil codes
    code = {
        1:2702,2:2703,3:2704,4:2705,5:2706,6:2717,7:3501,8:3502,9:4201,
        10:4703,11:4704,12:4744,13:4758,14:5101,15:5151,16:6101,17:6102,
        18:6731,19:7101,20:7102,21:7103,22:7201,23:7202,24:7700,25:7701,
        26:7702,27:7709,28:7710,29:7745,30:7746,31:7755,32:7756,33:7757,
        34:7790,35:8703,36:8707,37:8708,38:8771,39:8772,40:8776
    }
    
    # Climatic Zone
    df['Climatic_Zone'] = df['Soil_Type'].apply(lambda x: int(str(code[x])[0]))
    
    # Geologic Zone
    df['Geologic_Zone'] = df['Soil_Type'].apply(lambda x: int(str(code[x])[1]))
    
    # Surface Cover
    no_desc = [7,8,14,15,16,17,19,20,21,23,35]
    stony = [6,12]
    very_stony = [2,9,18,26]
    extremely_stony = [1,22,24,25,27,28,29,30,31,32,33,34,36,37,38,39,40]
    rubbly = [3,4,5,10,11,13]
    surface_cover = {i:0 for i in no_desc}
    surface_cover.update({i:1 for i in stony})
    surface_cover.update({i:2 for i in very_stony})
    surface_cover.update({i:3 for i in extremely_stony})
    surface_cover.update({i:4 for i in rubbly})
    
    df['Surface_Cover'] = df['Soil_Type'].apply(lambda x: surface_cover[x])

    # Rock Size
    no_desc = [7,8,14,15,16,17,19,20,21,23,35]
    stones = [1,2,6,9,12,18,24,25,26,27,28,29,30,31,32,33,34,36,37,38,39,40]
    boulders = [22]
    rubble = [3,4,5,10,11,13]
    rock_size = {i:0 for i in no_desc}
    rock_size.update({i:1 for i in stones})
    rock_size.update({i:2 for i in boulders})
    rock_size.update({i:3 for i in rubble})
    
    df['Rock_Size'] = df['Soil_Type'].apply(lambda x: rock_size[x])

            
    # Wilderness Interactions
    df['Climate_Area1'] = df['Wilderness_Area1']*df['Climatic_Zone'] 
    df['Climate_Area2'] = df['Wilderness_Area2']*df['Climatic_Zone'] 
    df['Climate_Area3'] = df['Wilderness_Area3']*df['Climatic_Zone'] 
    df['Climate_Area4'] = df['Wilderness_Area4']*df['Climatic_Zone'] 
    df['Geologic_Area1'] = df['Wilderness_Area1']*df['Geologic_Zone'] 
    df['Geologic_Area2'] = df['Wilderness_Area2']*df['Geologic_Zone']  
    df['Geologic_Area3'] = df['Wilderness_Area3']*df['Geologic_Zone'] 
    df['Geologic_Area4'] = df['Wilderness_Area4']*df['Geologic_Zone'] 
    df['Rock_Area1'] = df['Wilderness_Area1']*df['Rock_Size'] 
    df['Rock_Area2'] = df['Wilderness_Area2']*df['Rock_Size']   
    df['Rock_Area3'] = df['Wilderness_Area3']*df['Rock_Size']  
    df['Rock_Area4'] = df['Wilderness_Area4']*df['Rock_Size']
    df['Surface_Area1'] = df['Wilderness_Area1']*df['Surface_Cover'] 
    df['Surface_Area2'] = df['Wilderness_Area2']*df['Surface_Cover']   
    df['Surface_Area3'] = df['Wilderness_Area3']*df['Surface_Cover']  
    df['Surface_Area4'] = df['Wilderness_Area4']*df['Surface_Cover'] 
    df['Soil29_Area1'] = df['Soil_Type29'] + df['Wilderness_Area1']
    df['Soil3_Area4'] = df['Wilderness_Area4'] + df['Soil_Type3']
    
    # Drop redundant Soil columns
    df.drop(soil_features, axis = 1, inplace = True)
    
    # Fill NA
    df.fillna(0, inplace = True)
    
    # Downcast variables
    for col, dtype in df.dtypes.iteritems():
        if dtype.name.startswith('int'):
            df[col] = pd.to_numeric(df[col], downcast ='integer')
        elif dtype.name.startswith('float'):
            df[col] = pd.to_numeric(df[col], downcast ='float')
    
    return df

## Training

In [7]:
%%time
extratrees_features = train_original(
    extratrees, 
    feature_engineering
)

Wall time: 37min 32s


# Summary 

We test validation f1 scores and test accuracies for each feature vs the baseline

## Cross-Validation Accuracy

In [8]:
# ExtraTrees
pd.DataFrame.from_records(
    data = extratrees_features,
    columns = ['feature','cv_score','holdout','f1_score','time']
).sort_values('cv_score')

,feature,cv_score,holdout,f1_score,time
8,Elev_Binned,0.880813,0.773858,0.879319,34.788370
10,Hydro_Euclid,0.881078,0.777424,0.879715,34.624312
14,Hillshade_Range,0.882532,0.779301,0.881178,36.724407
53,Soil29_Area1,0.882996,0.782736,0.881735,32.477684
3,Aspect_360,0.883062,0.779743,0.881811,38.308810
23,EViElv,0.883128,0.777373,0.881815,36.239772
9,Hydro_Taxicab,0.883128,0.781114,0.881699,35.059199
48,Rock_Area4,0.883194,0.782409,0.881908,34.752842
4,Aspect_Sine,0.883194,0.779828,0.881876,36.161046
31,Elev_Hillshade_1,0.883393,0.774512,0.881919,33.220290


## ExtraTrees (F1 Score)

In [9]:
# ExtraTrees
pd.DataFrame.from_records(
    data = extratrees_features,
    columns = ['feature','cv_score','holdout','f1_score','time']
).sort_values('f1_score')

,feature,cv_score,holdout,f1_score,time
8,Elev_Binned,0.880813,0.773858,0.879319,34.788370
10,Hydro_Euclid,0.881078,0.777424,0.879715,34.624312
14,Hillshade_Range,0.882532,0.779301,0.881178,36.724407
9,Hydro_Taxicab,0.883128,0.781114,0.881699,35.059199
53,Soil29_Area1,0.882996,0.782736,0.881735,32.477684
3,Aspect_360,0.883062,0.779743,0.881811,38.308810
23,EViElv,0.883128,0.777373,0.881815,36.239772
4,Aspect_Sine,0.883194,0.779828,0.881876,36.161046
48,Rock_Area4,0.883194,0.782409,0.881908,34.752842
31,Elev_Hillshade_1,0.883393,0.774512,0.881919,33.220290


## Extra Trees (Holdout)

In [10]:
# ExtraTrees
pd.DataFrame.from_records(
    data = extratrees_features,
    columns = ['feature','cv_score','holdout','f1_score','time']
).sort_values('holdout')

,feature,cv_score,holdout,f1_score,time
8,Elev_Binned,0.880813,0.773858,0.879319,34.788370
31,Elev_Hillshade_1,0.883393,0.774512,0.881919,33.220290
32,Elev_Hillshade_2,0.883525,0.775952,0.882111,33.598643
23,EViElv,0.883128,0.777373,0.881815,36.239772
10,Hydro_Euclid,0.881078,0.777424,0.879715,34.624312
22,EHiElv,0.886568,0.778493,0.885240,36.024626
15,Hillshade,0.884451,0.779027,0.883124,37.020897
13,Hillshade_Avg,0.884451,0.779027,0.883124,36.053761
14,Hillshade_Range,0.882532,0.779301,0.881178,36.724407
5,Aspect_Alt,0.884055,0.779578,0.882789,34.723608
